In [1]:
import sys
import os
from os.path import join
from os import listdir as ld
from pathlib import Path
sys.path.append(join(Path(os.getcwd()).parent, 'lib'))

In [2]:
%matplotlib inline

import json
import os
import pandas as pd
import re
import numpy as np
import pickle
import collections
from importlib import reload
from  mainlib import fasorial, lista, sdict, mm, mm_sep
from unidecode import unidecode
from of_proc import *
from os.path import join
from os import listdir as ld
from unidecode import unidecode
from matplotlib import pyplot as plt

Using TensorFlow backend.


# Informações

In [3]:
def dk(d,k=0): return list(d.values())[k]

In [4]:
def read_info(path = '../../info/info.json'):
    global PATH, OUTPUT_PATH, PROCESSED_PATH, PERFIS_PATH, HORARIOS_PATH, PATH_MM, PATH_FAS, \
    DEG, MOR, PER, FAS, OUTPUT_HEADER,FAS_CONCAT, MM_CONCAT, PICKLES_CONCAT, OUT_INDEX, COLS_LISTA
    with open(path) as f:
        info = json.load(f)
        PATH_FAS = info['paths']['fasoriais2']['input']
        PERFIS_PATH = info['paths']['perfis']['perfis']
        HORARIOS_PATH = info['paths']['perfis']['horarios']
        PATH_MM = info['paths']['mm']['input']
        DEG = info['paths']['nn_models']['degrau']
        MOR = info['paths']['nn_models']['morro']
        PER = info['paths']['nn_models']['perfil']
        FAS = info['paths']['nn_models']['fasorial']
        COLS_LISTA = info['lista2']['columns']  
        OUTPUT_HEADER = info['output']['header']
        OUT_INDEX = info['output']['header']
read_info()
OUT = '../outputs/all/'
DATA_DIR = '/home/ubuntu/data/data/maio_2019/setembro/'
LISTA = join(DATA_DIR,'lista-cad-pnt-20190820.csv')

In [5]:
def get_horario_info2():
    with open('../../info/info.json') as jinfo:
        info = json.load(jinfo)
        HORARIOS_PATH = info['paths']['perfis']['horarios']
        PERFIS_PATH = '/home/ubuntu/data/data/maio_2019/setembro/perfis_v3.csv'
        ATIV_PATH = info['paths']['perfis']['mapa']


    perfis = pd.read_csv(PERFIS_PATH, sep = ';')
    ativs = pd.read_csv(ATIV_PATH, sep = ';')

    #perfis.columns = ['id', 'instalacao', 'cpf_cnpj', 'razao social', 'endereco_cadastro', 'ramo_de_atividade','None1', 'None2']
    
    dates = re.compile("[0-9]{2}:[0-9]{2}")
    dateparser = lambda x: pd.datetime.strptime(x, '%H:%M') if type(x) == str  and dates.match(x) else pd.NaT
    cols = ['name','inicio_dia_util','fim_dia_util', 'inicio_sabado', 'fim_sabado', 'inicio_domingo', 'fim_domingo', 'inicio_feriado', 'fim_feriado']
    horarios = pd.read_csv(HORARIOS_PATH, sep = ';',skipinitialspace=True, skiprows = 2, na_values = np.NaN, names = cols,
                          date_parser = dateparser, parse_dates = [1,2,3,4,5,6,7,8])
    
    
    horarios.set_index('name', inplace = True)
    horarios.dropna(inplace=True)

    perfis.set_index('instalacao', inplace=True)
    perfis = (perfis.groupby(level=0).first())

    
    return horarios, perfis, ativs

def get_files_4(nomes, dim,  CAMINHO):
    not_found = []
    dif = {}
    for i in nomes:
        if i in dim:
            tmp_list = []
            search = ab.search_list_2(str(i),os.listdir(CAMINHO))
            if search:
                dif[i] = search
            else:
                not_found.append(i)                
    return dif, not_found

#Criar dicionario de categorias
def insta2horario(HORARIOS_PATH = '/home/ubuntu/data/data/maio_2019/horarios_v4.csv'):
    horarios = pd.read_csv(HORARIOS_PATH, sep=',')
    horarios.drop(columns=horarios.columns[0],inplace=True)
    horarios.columns = ['nome', 'cat', 'dias_uteis_i',
           'dias_uteis_f', 'sabado_i', 'sabado_f', 'domingo_i',
           'domingo_f', 'feriado_i', 'feriado_f']
    horarios['nome'] = horarios['nome'].apply(unidecode)
    dfcat = horarios.iloc[:,:2]
    dfcat.set_index('nome',inplace=True)
    scat = dfcat['cat']
    dcat = scat.to_dict()
    horarios.set_index('nome', inplace=True)
    func = {k: [pd.Timestamp(v) for v in horarios.loc[k,'dias_uteis_i':'feriado_f']] for k in horarios.index}
    return func

def per2cluster(func, ATIV = '/home/ubuntu/data/data/maio_2019/atividade_atividade_v2.csv'):
    dfativ = pd.read_csv(ATIV, sep=';', names = ['ramo', 'perfil'])
    dfativ=dfativ.applymap(unidecode)
    dfativ.set_index('ramo', inplace=True)
    a2p = dfativ.loc[:,'perfil'].T.to_dict()
    a2p.pop('Ramo de Atividade')
    p2a = dict((v,k) for k,v in a2p.items())
    perfunc=PERFIS['ramo_de_atividade'].map(a2p).map(func)
    return perfunc

In [6]:
PATH_MM

'/home/ubuntu/data/data/maio_2019/setembro/mm/csv_proc/#pd_mmproc.csv'

In [7]:
#hc
PATH_MM='/home/ubuntu/data/data/maio_2019/setembro/mm/csv_proc/#pd_mmproc_comp.csv'

In [8]:
global df_mm_new
mmm = mm()
df_mm_new = pd.read_csv(PATH_MM,date_parser = mmm.dateparser, index_col=[0], sep=';')
dummy_mm = ""

## l_______________________________________________________________________

### Carregar modelos neurais

# Coleções principais

In [9]:
global LISTA_CAD
global PERFIL, DEGRAU, MORRO, NN_FAS
global files_fas, files_mm
global HORARIOS, PERFIS, ATIVS, DPERFIS, DPERFIS_NORMAL, DPERFIS_TROCA
global dim, dmi, df_fas, dfiles_mm, dfiles_fas_troca, dfiles_mm_troca, dfiles_fas_normal, dfiles_mm_normal
global o_i_normal, o_i_troca, o_instas, list_i_normal, list_i_troca, list_intas

#### Modelos neurais

In [10]:
#hc
PER = '../../models/r2-DS110k-bs64-c11/r2-DS110k-bs64-c11'
DEG = '../../models/degrau/degrau'
MOR = '../../models/anomalia-1_1_v2/anomalia-1_1_v2'
FAS = '../../models/rn_hc_and/rn_hc_and'

In [11]:
PERFIL, DEGRAU, MORRO, NN_FAS = load_models(PER, DEG, MOR, FAS)

Instructions for updating:
Colocations handled automatically by placer.


## l_______________________________________________________________________

In [12]:
PATH_FAS

'/home/ubuntu/data/data/maio_2019/setembro/fas-sep'

In [13]:
#hc
LISTAF = '/home/ubuntu/data/data/maio_2019/setembro/lista-cad-pnt-20190820-faltantes.csv'

In [14]:
LISTA_CAD = get_list()
LISTA_CAD.df =  pd.read_csv(LISTA,names = COLS_LISTA)
#LISTA_CAD.df =  pd.read_csv(LISTAF,names = COLS_LISTA)
HORARIOS, PERFIS, ATIVS = get_horario_info2()
dim, dmi = dicts(LISTA_CAD)

-------------------------------

#### Verificar

###### Lista de instalações faltantes

In [15]:
fasfal4 = pload('../../pickles/fasfal4.pkl')

##### Verificar se faltantes estão na lista 1

In [16]:
l = pd.read_csv(LISTA,names = COLS_LISTA).drop(0)
lo = [i for i in fasfal4 if i  in l['cliente']]
len(lo)

0

##### Verificar se faltantes estão na lista 2

In [69]:
l2 = pd.read_csv(LISTAF,names = COLS_LISTA)
lo2 = [i for i in fasfal4 if i  in l2['cliente']]
len(lo2)

0

##### Verificar se faltantes estão entre os arquivos separados

In [70]:
fl = [i.split('.')[0] for i in ld(PATH_FAS)]
lo2  = [i for i in fasfal4 if i  in fl]
len(lo2)

20

-------------------------------

#### Estruturas auxiliares

In [17]:
o_i_normal = collections.OrderedDict(sorted(LISTA_CAD.mes_days(troca=False).items()))
o_i_troca = collections.OrderedDict(sorted(LISTA_CAD.mes_days(troca=True).items()))
o_instas =  collections.OrderedDict(sorted({**o_i_normal, **o_i_troca}.items()))

#### Arquivos

In [18]:
import A_base
reload(A_base)

<module 'A_base' from '/home/ubuntu/data/code/ref_fdetector/src/lib/A_base.py'>

In [19]:
import A_base as ab

In [20]:
dfiles_fas, nf_fas= get_files_4(list(o_instas.keys()), dim,  PATH_FAS)
ddirs_fas = {k:join(PATH_FAS,v) for k,v in dfiles_fas.items()}

 l___________________________________________________________________________________________________________________________________________

In [21]:
pdump(ddirs_fas, '../../pickles/ddirs_fas.pkl')

 l___________________________________________________________________________________________________________________________________________

#### Lista , perfis e horários

In [22]:
# Dicionário de instalacao para horario de funcionamento
func =  insta2horario()

In [23]:
# Dicionário de perfis para clusters
perfunc = per2cluster(func)

In [24]:
percats = PERFIS['cat'].fillna(value=0).apply(int).to_dict()

 l___________________________________________________________________________________________________________________________________________

#### Pickles

## l_______________________________________________________________________

In [25]:
lista_falta = [
'BTI413081598',
'BTI420465568',
'BTI411793194',
'BTI411211744',
'BTI410364472',
'BTI411156078',
'BTI412687778',
'BTI410049617',
'BTI400114910',
'BTI410447700',
'BTI411659603',
'BTI412787198',
'BTI411837868',
'BTI410008428',
'BTI410364444',
'BTI400085863',
'BTI411999140',
'BTI411387076',
'BTI411177604',
'BTI414581058',
'BTI400086150',
'BTI410732088',
'BTI420639963',
'BTI410426420',
'BTI420201181',
'BTI410758477',
'BTI411653674',
'BTI420486949',
'BTI413595836',
'BTI420569650',
'BTI420738351',
'BTI412003978',
'BTI411574674',
'BTI411284122',
'BTI410328848',
'BTI411212425',
'BTI412171105',
'BTI412513987',
'BTI400086681',
'BTI420892596',
'BTI413860891',
'BTI420318146',
'BTI411878714',
'BTI410155805',
'BTI412487479',
'BTI400091995',
'BTI411837862',
'BT2A410913928',
'BT2A411456287',
'BT2A413325780',
'BT2A413642837',
'BT2A420515786',
'BT2A400300110',
'BT2A414604945',
'BT2A420135273',
'BT2A411350480',
'BT2A413741641',
'BT2A411653104',
'BT2A411071225',
'BT2A420700602',
'BT2A411145184',
'BT2A410476676',
'BTI412034558',
'BTI413648141',
'BTI410522512',
'BTI411055611',
'BTI411828439',
'BTI420811160',
'BTI413267453',
'BTI421042017',
'BTI400153567',
'BTI412191537',
'BTI413446145',
'BTI413497182',
'BTI414692427',
'BTI413273677',
'BTI411175463',
'BTI413339942',
'BTI420165364',
'BTI400064734',
'BTI411287978'
]

In [28]:
lista_falta.remove('BT2A411350480')
lista_falta.remove('BTI411177604') 
lista_falta.remove('BTI421042017')

## l_______________________________________________________________________

# Importante

In [30]:
log_count=0

In [31]:
processados = []
erros = []

In [32]:
sem_hora = []
err_msg = {}
missing = []

In [33]:
with open('../../pickles/all/processados.pkl','rb') as p:
    processados=pickle.load(p)
with open('../../pickles/all/erros.pkl','rb') as p:
    erros = pickle.load(p)

In [34]:
log_file = open(os.path.join('../../logs/',str(log_count)), 'at+')

In [35]:
def log(string):
    print(string)
    log_file.write(string)

In [36]:
def dici2list(d_insta_to_days):
    ddays={}
    ddays[4],ddays[5],ddays[6],ddays[7] = d_insta_to_days[12], d_insta_to_days[1], d_insta_to_days[2], d_insta_to_days[3], d_insta_to_days[4]
    return ddays
                

In [37]:
import traceback

In [38]:
pd.options.mode.chained_assignment = None  # default='warn' . option None

In [39]:
padrao = [pd.Timestamp('2019-09-20 00:00:00'),
 pd.Timestamp('2019-09-20 23:59:00'),
 pd.Timestamp('2019-09-20 00:00:00'),
 pd.Timestamp('2019-09-20 23:59:00'),
 pd.Timestamp('2019-09-20 00:00:00'),
 pd.Timestamp('2019-09-20 23:59:00'),
 pd.Timestamp('2019-09-20 00:00:00'),
 pd.Timestamp('2019-09-20 23:59:00')]

In [40]:
def loopy2(d_insta_to_days, d_insta_to_files_fas, d_insta_to_files_mm,PATH_FAS, PATH_MM, output_dir,sepfas=';', sepmm = ','):
    df_out = pd.DataFrame(index=OUT_INDEX)
    global difas, dimm
    global dfas, dmm,  dimm
    difas, dimm = sdict(), sdict()
    c = 0
    c1 = 0
    for instalacao in d_insta_to_days:
        c1 = c1 + 1
        if (instalacao  not in processados):
            log(str(c)+'--'+'######## ' + instalacao + ' #########')
            medidor = dim[instalacao][0] if type(dim[instalacao]) is list else dim[instalacao]
            #if True:
            try:      
                print('------ 0')
                ramo = percats[instalacao] if instalacao in percats else 0
                print('------ 1')
                hours = perfunc[instalacao] if instalacao in perfunc else padrao
                print('------ 2')
                ddays = d_insta_to_days[instalacao] if instalacao in d_insta_to_days else default_ddays_2()
                print('------ 3')
                dfas = fasorial(name = instalacao, ddays=ddays)     
                print('------ 4')
                dfas.outputs['perfil'] = ramo
                print('------ 5')
                dfas.read_fasorial(d_insta_to_files_fas[instalacao], dtype=None, index_col=[5], sep = sepfas)
                print('------ 6')
                dmm = mm(name = instalacao, ddays = ddays, ramo=ramo)
                print('------ 7')
                dmm.df = pd.DataFrame(df_mm_new[int(dim[instalacao])])  
                print('------ 8')
                proc_fas(dfas,ddays, NN_FAS, hours = dfas.hours)
                processa_mm(dmm, ddays, PERFIL, DEGRAU, MORRO)
                print('################################################')
                #Cria um dicionário de outputs por mês para o fasorial
                for y in dfas.months:
                    dfas.outputs[y] = {}
                    for month  in dfas.months[y]:
                        dfas.outputs[y][month] = dfas.months[y][month].outputs                
                df_out = cat_out2(df_out, dmm.outputs, dfas.outputs, ddays,OUT_INDEX)
                
                difas[instalacao] = dfas
                dimm[instalacao] = dmm                
                c = c+1                
                #if c1 % 100 ==0 :  
                if False:
                    #return df_out, dfas, dmm
                    log('Salvando csv!')
                    df_out.T.to_csv(output_dir + 'e1_' +str(c1-100) + '_to_' + str(c1) +'.csv')                    
                    pickle.dump(processados, open('../pickles/all/processados.pkl','wb'))
                    pickle.dump(erros, open('../pickles/all/erros.pkl','wb'))                    
                    df_out=pd.DataFrame()
                processados.append(instalacao)
            except Exception as e:
            #else:
                erros.append(instalacao)
                err_msg[instalacao] = traceback.format_exc()
                print('++++++Erro 2++++++++++ \n'+ str(traceback.format_exc())+'\n++++++++++++++++++++')
    #return outs,df_out, dfas, dmm
    #pickle.dump(processados, open('../../pickles/all/processados.pkl','wb'))
    #pickle.dump(erros, open('../../pickles/all/erros.pkl','wb'))
    df_out.T.to_csv(output_dir + 'new.csv')        

## l_______________________________________________________________________

In [41]:
er = {k:v for k,v in o_instas.items() if k in lista_falta}

In [42]:
from mainlib import mm

In [43]:
OUT = '../../outputs/all/'

## l_______________________________________________________________________

In [44]:
df_mm_new.columns = [int(float(c)) for c in df_mm_new.columns]

## l_______________________________________________________________________

In [45]:
processados, erros = [], []
m=loopy2(er, ddirs_fas, dummy_mm,PATH_FAS, PATH_MM,OUT,sepmm=',')

0--######## BT2A400300110 #########
------ 0
------ 1
------ 2
------ 3
------ 4
------ 5
------ 6
------ 7
------ 8
{5: Timestamp('2019-05-06 00:00:00'), 6: Timestamp('2019-06-06 00:00:00'), 7: Timestamp('2019-07-04 00:00:00')}


/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:677: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]
/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:677: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]
/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:677: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]


{5: Timestamp('2019-05-06 00:00:00'), 6: Timestamp('2019-06-06 00:00:00'), 7: Timestamp('2019-07-04 00:00:00')}
++++++Erro 2++++++++++ 
Traceback (most recent call last):
  File "<ipython-input-40-854869aae9c4>", line 33, in loopy2
    processa_mm(dmm, ddays, PERFIL, DEGRAU, MORRO)
  File "/home/ubuntu/data/code/ref_fdetector/src/lib/of_proc.py", line 340, in processa_mm
    mm.nn_perfil(imodel = perfil)
  File "/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py", line 1030, in nn_perfil
    self.outputs['perfil'][y][m] = self.months[y][m].nn_perfil_month(imodel)
  File "/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py", line 1042, in nn_perfil_month
    preds[c] = model.predict(np.asarray(week).reshape(1,-1))
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py", line 1149, in predict
    x, _, _ = self._standardize_user_data(x)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/tra

/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:677: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]
/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:677: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]


{5: Timestamp('2019-05-21 00:00:00'), 6: Timestamp('2019-06-22 00:00:00'), 7: Timestamp('2019-07-22 00:00:00')}
################################################
1--######## BT2A410913928 #########
------ 0
------ 1
------ 2
------ 3
------ 4
------ 5
------ 6
------ 7
------ 8
{5: Timestamp('2019-05-06 00:00:00'), 6: Timestamp('2019-06-06 00:00:00'), 7: Timestamp('2019-07-04 00:00:00')}


...

/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:677: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]


{5: Timestamp('2019-05-27 00:00:00'), 6: Timestamp('2019-06-26 00:00:00'), 7: Timestamp('2019-07-25 00:00:00')}
################################################
60--######## BTI420811160 #########
------ 0
------ 1
------ 2
------ 3
------ 4
------ 5
++++++Erro 2++++++++++ 
Traceback (most recent call last):
  File "<ipython-input-40-854869aae9c4>", line 26, in loopy2
    dfas.read_fasorial(d_insta_to_files_fas[instalacao], dtype=None, index_col=[5], sep = sepfas)
KeyError: 'BTI420811160'

++++++++++++++++++++
60--######## BTI420892596 #########
------ 0
------ 1
------ 2
------ 3
------ 4
------ 5
------ 6
------ 7
------ 8
{5: Timestamp('2019-05-17 00:00:00'), 6: Timestamp('2019-06-19 00:00:00'), 7: Timestamp('2019-07-18 00:00:00')}


/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:677: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]
/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:677: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]


{5: Timestamp('2019-05-17 00:00:00'), 6: Timestamp('2019-06-19 00:00:00'), 7: Timestamp('2019-07-18 00:00:00')}
################################################


/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:677: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]


In [206]:
processados, erros = [], []
m=loopy2(o_instas, ddirs_fas, dummy_mm,PATH_FAS, PATH_MM,OUT,sepmm=',')

0--######## BT2A410315584 #########
------ 0
------ 1
------ 2
------ 3
------ 4
------ 5
------ 6
------ 7
------ 8
{5: Timestamp('2019-05-28 00:00:00'), 6: Timestamp('2019-06-27 00:00:00'), 7: Timestamp('2019-07-26 00:00:00')}


/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:676: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]
/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:676: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]
/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:676: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]


{5: Timestamp('2019-05-28 00:00:00'), 6: Timestamp('2019-06-27 00:00:00'), 7: Timestamp('2019-07-26 00:00:00')}
################################################
1--######## BT2A410444087 #########
------ 0
------ 1
------ 2
------ 3
------ 4
------ 5
------ 6
------ 7
------ 8
{5: Timestamp('2019-05-28 00:00:00'), 6: Timestamp('2019-06-27 00:00:00'), 7: Timestamp('2019-07-26 00:00:00')}


/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:676: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]
/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:676: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]
/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:676: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]


{5: Timestamp('2019-05-28 00:00:00'), 6: Timestamp('2019-06-27 00:00:00'), 7: Timestamp('2019-07-26 00:00:00')}
################################################
2--######## BT2A410457345 #########
------ 0
------ 1
------ 2
------ 3
------ 4
------ 5
------ 6
------ 7
------ 8
{5: Timestamp('2019-05-28 00:00:00'), 6: Timestamp('2019-06-27 00:00:00'), 7: Timestamp('2019-07-26 00:00:00')}


...

/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:676: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]
/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:676: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]
/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:676: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]


{5: Timestamp('2019-05-27 00:00:00'), 6: Timestamp('2019-06-26 00:00:00'), 7: Timestamp('2019-07-25 00:00:00')}
################################################
91--######## BTI421155328 #########
------ 0
------ 1
------ 2
------ 3
------ 4
------ 5
------ 6
------ 7
------ 8
{5: Timestamp('2019-05-29 00:00:00'), 6: Timestamp('2019-06-28 00:00:00'), 7: Timestamp('2019-07-29 00:00:00')}


/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:676: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]
/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:676: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]


{5: Timestamp('2019-05-29 00:00:00'), 6: Timestamp('2019-06-28 00:00:00'), 7: Timestamp('2019-07-29 00:00:00')}
################################################


/home/ubuntu/data/code/ref_fdetector/src/lib/mainlib.py:676: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(f1 & f2) | f3]


In [168]:
df_mm_new.columns=pd.Series(df_mm_new.columns).apply(float).apply(int)

In [141]:
l = list(LISTA_CAD.df['instalacao'][1:])

In [147]:
df_mm_new.columns

Index(['BTI400085863', 'BTI400086150', 'BTI400086681', 'BTI400091995',
       'BTI400114910', 'BTI410008428', 'BTI410049617', 'BTI410155805',
       'BTI410328848', 'BTI410364444',
       ...
       'BT2A414490860', 'BT2A414509928', 'BT2A414642567', 'BT2A414865002',
       'BT2A420054001', 'BT2A420399343', 'BT2A420492569', 'BT2A421052825',
       'BT2A421122290', 'BT2A421250758'],
      dtype='object', length=1348)

In [145]:
i = [a for a in l if a in df_mm_new.columns]

In [195]:
perfunc.iloc[0]

[Timestamp('2019-09-20 00:00:00'),
 Timestamp('2019-09-20 23:59:00'),
 Timestamp('2019-09-20 00:00:00'),
 Timestamp('2019-09-20 23:59:00'),
 Timestamp('2019-09-20 00:00:00'),
 Timestamp('2019-09-20 23:59:00'),
 Timestamp('2019-09-20 00:00:00'),
 Timestamp('2019-09-20 23:59:00')]

## l_______________________________________________________________________

##### Substituir coluna "Perfil cadastrado" por texto

In [209]:
l=LISTA_CAD.df.drop(index=0)

In [214]:
ra=PERFIS['ramo_de_atividade']

In [217]:
dra=ra.to_dict()

In [222]:
o = pd.read_csv('../../outputs/all/new.csv')

In [237]:
o['Perfil cadastrado']=o['Nome'].map(dra)

## l_______________________________________________________________________

#### Leituras faltando - Fasorial

In [123]:
import mainlib
reload(mainlib)
from mainlib import dframe

dfr = dframe()
data = dfr.data
dp = dfr.dateparser

ddirs_fas = pload('../pickles/ddirs_fas.pkl')

emp = {}
for n,(k,v) in enumerate(ddirs_fas.items()):
    df = pd.read_csv(v,sep=';',date_parser = dp,parse_dates=[5],names=data['fasorial']['columns'])
    emp[k]=df.set_index('data').iloc[:,0].resample('15min').first().isna().sum()
    print(n,k,emp[k])

demp=pd.DataFrame(index=emp.keys(), data=emp.values(),columns = ['Leituras faltantes'])

In [123]:
demp.to_csv('../outputs/fas_emp.csv')

## l_______________________________________________________________________

#### Leituras faltando - Fasorial

In [258]:
PATH_MM

'/home/ubuntu/data/data/maio_2019/setembro/mm/csv_proc/#pd_mmproc.csv'

In [285]:
mm = pd.read_csv(PATH_MM, parse_dates = [0],date_parser=dp, index_col=[0])

In [289]:
mm[mm.index.isnull()]

,BTI400085863,BTI400086150,BTI400086681,BTI400091995,BTI400114910,BTI410008428,BTI410049617,BTI410155805,BTI410328848,BTI410364444,...,BT2A414490860,BT2A414509928,BT2A414642567,BT2A414865002,BT2A420054001,BT2A420399343,BT2A420492569,BT2A421052825,BT2A421122290,BT2A421250758
DATA/Hora,,,,,,,,,,,,,,,,,,,,,
NaT,317.69,18480.96,-300465.17,4272.71,14387.52,144.19,40532.66,10554.97,-133523.24,1492.69,...,20365.55,4714.32,6975.9,8566.44,6798.06,4886.19,2.25,3474.63,14747.29,14069.35


In [279]:
mm.isnull().sum().sum()

1

In [283]:
mm.describe()

,BTI400085863,BTI400086150,BTI400086681,BTI400091995,BTI400114910,BTI410008428,BTI410049617,BTI410155805,BTI410328848,BTI410364444,...,BT2A414490860,BT2A414509928,BT2A414642567,BT2A414865002,BT2A420054001,BT2A420399343,BT2A420492569,BT2A421052825,BT2A421122290,BT2A421250758
count,2953.000000,2953.000000,2953.000000,2953.000000,2953.000000,2953.000000,2953.000000,2953.000000,2953.000000,2953.000000,...,2953.000000,2953.000000,2953.000000,2953.000000,2953.000000,2953.000000,2953.000000,2953.000000,2953.000000,2953.000000
mean,0.215181,12.516739,-203.497176,2.893630,9.744328,0.097775,27.451859,7.148581,-90.431818,1.010918,...,13.793248,3.192845,4.724521,5.801863,4.604179,3.309326,0.001707,2.353329,9.988043,9.528920
std,5.847146,340.146631,5530.056625,78.628187,264.767265,2.653908,745.893431,194.271733,2457.672610,27.488508,...,374.699653,86.728285,128.379483,157.594868,125.060987,89.890404,0.042774,63.977131,271.305960,258.906227
min,0.000000,0.000000,-300465.170000,0.000000,0.000000,0.000000,0.000000,0.000000,-133523.240000,0.000000,...,0.000000,0.220000,0.000000,0.000000,0.120000,0.000000,0.000000,0.000000,0.000000,0.420000
25%,0.000000,0.000000,-393.020000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.990000,0.950000,0.130000,2.070000,1.560000,0.990000,0.000000,0.270000,3.950000,0.620000
50%,0.000000,2.370000,0.000000,0.940000,2.380000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.680000,1.400000,0.420000,2.460000,2.180000,1.320000,0.000000,0.450000,5.560000,0.740000
75%,0.270000,5.820000,1.150000,1.060000,4.640000,0.050000,25.310000,6.960000,0.000000,0.350000,...,12.360000,2.070000,3.970000,3.220000,3.170000,2.510000,0.000000,0.730000,6.340000,6.650000
max,317.690000,18480.960000,28.370000,4272.710000,14387.520000,144.190000,40532.660000,10554.970000,0.180000,1492.690000,...,20365.550000,4714.320000,6975.900000,8566.440000,6798.060000,4886.190000,2.250000,3474.630000,14747.290000,14069.350000
